In [ ]:
# Parser AWS transcript

In [1]:
import pandas as pd
import re

transcript = pd.read_csv('aws_transcript.csv')
df = transcript.copy()
df.head()

,Unnamed: 0,start_time,end_time,speaker_label,text
0,0,0.000150,0.025817,spk_0,I can record
1,1,0.046983,0.083983,spk_0,and we don't have a ton
2,2,0.096333,0.125650,spk_0,"of items to get to,"
3,3,0.132333,0.142000,spk_0,uh
4,4,0.154650,0.208983,spk_0,and I might be able to do one that might be fun


In [5]:
prev_text = ''
prev_speaker = ''
merge_start = 0.0

#removing ' and " from text using regex
df['text'] = df['text'].apply(lambda x: re.sub(r"[\"\'\--]+", '', x))

for index, row in df.iterrows():
    # Replace ' and "" with blank space
    if row['text'] == prev_text and row['speaker_label'] == prev_speaker:
        # Update end time for consecutive rows
        df.at[merge_start, 'end_time'] = row['end_time']
        df.drop(index, inplace=True)
    else:
        merge_start = index

    prev_text = row['text']
    prev_speaker = row['speaker_label']

In [6]:
df.head()

,Unnamed: 0,start_time,end_time,speaker_label,text
0,0,0.000150,0.025817,spk_0,I can record
1,1,0.046983,0.083983,spk_0,and we dont have a ton
2,2,0.096333,0.125650,spk_0,"of items to get to,"
3,3,0.132333,0.142000,spk_0,uh
4,4,0.154650,0.208983,spk_0,and I might be able to do one that might be fun


In [7]:
df['group'] = (df['speaker_label'] != df['speaker_label'].shift()).cumsum()
result_df = df.groupby(['group', 'speaker_label'], as_index=False).agg({'start_time': 'first', 'end_time': 'last', 'text': ' '.join})
result_df = result_df.drop(columns=['group'])


In [8]:
result_df.to_csv('aws_parsed_transcript.csv', index=False)

In [ ]:
def aws_transcribe_parser(transcript: pd.DataFrame) -> None:
    """
    Parses the AWS Transcribe output by deleting duplicate texts, and merging consecutive rows with the same speaker.
    """
    transcript['text'] = transcript['text'].apply(lambda x: re.sub(r"[\"\'\--]+", '', x))

    for index, row in transcript.iterrows():
        if row['text'] == prev_text and row['speaker_label'] == prev_speaker:
            transcript.at[merge_start, 'end_time'] = row['end_time']
            transcript.drop(index, inplace=True)
        else:
            merge_start = index

        prev_text = row['text']
        prev_speaker = row['speaker_label']

    transcript['group'] = (transcript['speaker_label'] != transcript['speaker_label'].shift()).cumsum()
    result_df = transcript.groupby(['group', 'speaker_label'], as_index=False).agg({'start_time': 'first', 'end_time': 'last', 'text': ' '.join})
    result_df = result_df.drop(columns=['group'])
    transcript = result_df.copy()    

aws_transcribe_parser(transcript)    